In [1]:
import pyedflib
import numpy as np
import xmltodict
import json
import mne
import matplotlib
import math
import pathlib
from mne_extras import write_mne_edf

In [2]:
matplotlib.use('Qt5Agg')

In [3]:
#setting up paths for inputs and outputs
file_name = '2021-07-13_152554_RawData'

input_path = './input_txt/mzk_20210629/'
output_path_edf = './output_edf/'
output_path_fif = './output_fif/'

ch_one = file_name + '_Ch1.txt'
ch_two = file_name + '_Ch2.txt'
# ch_one = '2021-07-09_094849_Filter_Data' + '_Ch1.txt'
# ch_two = '2021-07-09_094849_Filter_Data' + '_Ch2.txt'

ch_three = file_name + '_Ch3.txt'
ch_four = file_name + '_Ch4.txt'

xml = '2021-07-13_152554.xml'

out_file_name = 'singleCH_backhead'
out_name_edf = out_file_name + '.edf'
out_name_fif = out_file_name + '.fif'
out_name_final = 'final_' + out_file_name + '.edf'

ch_one_path = input_path + ch_one
ch_two_path = input_path + ch_two
ch_three_path = input_path + ch_three
ch_four_path = input_path + ch_four
xml_path = input_path + xml
out_file_path_edf = output_path_edf + out_name_edf
out_file_path_fif = output_path_fif + out_name_fif
out_final_edf = output_path_edf + out_name_final

In [4]:
#read from .txt and convert data into numpy array
#each dataset is composed of two channels (.txt) and one information doc (.xml)
raw_one = np.loadtxt(ch_one_path)
raw_two = np.loadtxt(ch_two_path)
raw_three = np.loadtxt(ch_three_path)
raw_four = np.loadtxt(ch_four_path)
signal = [np.array(raw_one, dtype=np.float32), np.array(raw_two, dtype=np.float32), np.array(raw_three, dtype=np.float32), np.array(raw_four, dtype=np.float32)]

In [5]:
#read .xml doc and extract needed info
fileptr = open(xml_path, "r")

xml_content = fileptr.read()

my_ordered_dict = xmltodict.parse(xml_content)
dict = json.loads(json.dumps(my_ordered_dict))

sample_rate = eval(dict['RECORD_INFO']['Record']['SamplesFreq'])

In [6]:
#setting up info needed for .edf generation and write .edf file
headers = [{'label':'ch1', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
            {'label':'ch2', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch3', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch4', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'}]
with open(out_file_path_edf, 'w') as output:
    print(out_file_path_edf)
    flag = pyedflib.highlevel.write_edf(output.name, signal, headers, header=None, digital=False, file_type=-1, block_size=1)
    print(flag)

./output_edf/singleCH_backhead.edf
True


In [7]:
#read the newly created .edf using mne
raw=mne.io.read_raw_edf(out_file_path_edf,preload=False)

Extracting EDF parameters from C:\Users\admin\Desktop\work\my_evaluator\output_edf\singleCH_backhead.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [8]:
raw

<RawEDF | singleCH_backhead.edf, 4 x 127875 (125.0 s), ~9 kB, data not loaded>

In [9]:
raw.plot()

<MNEBrowseFigure size 1918x784 with 4 Axes>

In [10]:
#create events using mne
#events are equally spaced out for epoch division
new_events = mne.make_fixed_length_events(raw, duration=2.)
event_dict = {'divide':1}

In [11]:
#filter using frequency
#best: do after the rejection, but will reject 89% of data
raw.load_data()
raw.filter(l_freq=0.5, h_freq=30)

Reading 0 ... 127874  =      0.000 ...   124.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6753 samples (6.601 sec)



<RawEDF | singleCH_backhead.edf, 4 x 127875 (125.0 s), ~3.9 MB, data loaded>

In [12]:
#reject using values of amplitude
#best: do before the filter, but will reject 89% of data
del (dict)
reject_criteria = dict(eeg=400e-6)       # 400 µV

flat_criteria = dict(eeg=1e-6)           # 1 µV

epochs = mne.Epochs(raw,new_events, reject=reject_criteria, flat=flat_criteria,
                    reject_by_annotation=False, preload=True, picks=['ch1','ch2'])

Not setting metadata
Not setting metadata
62 matching events found
Setting baseline interval to [-0.20039100684261973, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 62 events and 718 original time points ...
1 bad epochs dropped


In [13]:
epochs_2 = mne.Epochs(raw,new_events, reject_by_annotation=False, preload=True)

Not setting metadata
Not setting metadata
62 matching events found
Setting baseline interval to [-0.20039100684261973, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 62 events and 718 original time points ...
1 bad epochs dropped


In [14]:
epochs.plot_drop_log()

Channels marked as bad: none


<Figure size 640x480 with 1 Axes>

In [15]:
# epochs.load_data()
# epochs.filter(l_freq=0.5, h_freq=30)

In [16]:
epochs.plot_psd(average=True)

    Using multitaper spectrum estimation with 7 DPSS windows


<MNELineFigure size 1000x350 with 1 Axes>

In [17]:
epochs.plot()
epochs_2.plot()

<MNEBrowseFigure size 1918x784 with 4 Axes>

In [18]:
#https://mne.tools/stable/auto_tutorials/preprocessing/50_artifact_correction_ssp.html#tut-artifact-ssp

#cannot do ecg filter because we dont have any ecg channel or meg channel
#ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=0, n_mag=0, n_eeg=4, ch_name='ch1', reject = None)
eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=0, n_mag=0, n_eeg=1, ch_name='ch1', reject = None)

Including 0 SSP projectors from raw file
Running EOG SSP computation
Using EOG channel: ch1
EOG channel index for this subject is: [0]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10230 samples (10.000 sec)

Now detecting blinks and generating corresponding events
Found 9 significant peaks
Number of EOG events detected: 9
Computing projector
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
---------------------
Designing a two-pass forward a

In [19]:
projs = eog_projs
#projs = eog_projs + ecg_projs

In [20]:
epochs.add_proj(projs)

1 projection items deactivated


Number of events,61
Events,1: 61
Time range,-0.200 – 0.500 sec
Baseline,-0.200 – 0.000 sec


In [21]:
#apply eog filter
epochs_cleaned = epochs.copy().apply_proj()

Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


C:\Users\admin\AppData\Local\Temp/ipykernel_12228/1745490712.py:2: RuntimeWarning: Projection vector "EOG-eeg--0.200-0.200-PCA-01" has magnitude 1.00 (should be unity), applying projector with 2/4 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  epochs_cleaned = epochs.copy().apply_proj()


In [22]:
epochs[0:100].plot()
epochs_cleaned[0:100].plot()

C:\Users\admin\AppData\Local\Temp/ipykernel_12228/2543344250.py:2: RuntimeWarning: Projection vector "EOG-eeg--0.200-0.200-PCA-01" has magnitude 1.00 (should be unity), applying projector with 2/4 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  epochs_cleaned[0:100].plot()


<MNEBrowseFigure size 1918x784 with 4 Axes>

In [23]:
#save as .fif format
epochs.save(out_file_path_fif, overwrite=True)

C:\Users\admin\AppData\Local\Temp/ipykernel_12228/2457816835.py:2: RuntimeWarning: This filename (./output_fif/singleCH_backhead.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_file_path_fif, overwrite=True)


In [24]:
#convert to data frame format in order to save as .edf
df = epochs.to_data_frame()

In [25]:
#save as .edf
out_raw_one = df['ch1']
out_raw_two = df['ch2']
# out_raw_three = df['ch3']
# out_raw_four = df['ch4']
#out_signal = [np.array(out_raw_one), np.array(out_raw_two), np.array(out_raw_three), np.array(out_raw_four)]
out_signal = [np.array(out_raw_one), np.array(out_raw_two)]

headers = [{'label':'ch1', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
            {'label':'ch2', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'}]

with open(out_final_edf, 'w') as output:
    print(out_final_edf)
    flag = pyedflib.highlevel.write_edf(output.name, out_signal, headers, header=None, digital=False, file_type=-1, block_size=1)
    print(flag)

./output_edf/final_singleCH_backhead.edf
True


In [26]:
variance = np.var(out_signal)
print('variance:', variance)


variance: 185.9692319696723


In [27]:
epochs.plot_psd(average=True, fmin=0., fmax=100)

    Using multitaper spectrum estimation with 7 DPSS windows
Dropped 0 epochs: 
Channels marked as bad: none
Dropped 0 epochs: 
Channels marked as bad: none
Dropped 0 epochs: 
Channels marked as bad: none
Dropped 0 epochs: 
Channels marked as bad: none


<MNELineFigure size 1000x350 with 1 Axes>

In [28]:
psds_total, frqs_total = mne.time_frequency.psd_multitaper(epochs, fmin=0.5, fmax=44.5, tmin=None, tmax=None)
total_sum_pds = np.sum(psds_total)
print(total_sum_pds)

psds_power, frqs_power = mne.time_frequency.psd_multitaper(epochs, fmin=45, fmax=55, tmin=None, tmax=None)
power_sum_pds = np.sum(psds_power)
print(power_sum_pds)
power_ratio = (power_sum_pds) / (total_sum_pds)
print(power_ratio)
print(frqs_power)

    Using multitaper spectrum estimation with 7 DPSS windows
1.3154165158816441e-05
    Using multitaper spectrum estimation with 7 DPSS windows
3.003908922243861e-09
0.00022836180677194269
[45.59331476 47.01810585 48.44289694 49.86768802 51.29247911 52.71727019
 54.14206128]


In [29]:
# psds_total, frqs_total = mne.time_frequency.psd_welch(epochs, fmin=0.1, fmax=100, tmin=None, tmax=None, n_fft = 718)
# total_sum_pds = np.sum(psds_total)
# print(total_sum_pds)
# psds_part, frqs_part = mne.time_frequency.psd_welch(epochs, fmin=45, fmax=55, tmin=None, tmax=None, n_fft = 718)
# part_sum_pds = np.sum(psds_part)
# print(part_sum_pds)
# ratio = (part_sum_pds) / (total_sum_pds)
# print(ratio)
# print(frqs_part)

In [30]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#delta
print(total_sum_pds)
psds_delta, frqs_delta = mne.time_frequency.psd_multitaper(epochs, fmin=1.5, fmax=4.5, tmin=None, tmax=None)
delta_sum_pds = np.sum(psds_delta)
print(delta_sum_pds)
delta_ratio = (delta_sum_pds) / (total_sum_pds)
print(delta_ratio)
print(frqs_delta)

1.3154165158816441e-05
    Using multitaper spectrum estimation with 7 DPSS windows
1.5779902085359334e-06
0.1199612586191608
[2.84958217 4.27437326]


In [31]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#theta
print(total_sum_pds)
psds_theta, frqs_theta = mne.time_frequency.psd_multitaper(epochs, fmin=4.5, fmax=7.5, tmin=None, tmax=None)
theta_sum_pds = np.sum(psds_theta)
print(theta_sum_pds)
theta_ratio = (theta_sum_pds) / (total_sum_pds)
print(theta_ratio)
print(frqs_theta)

1.3154165158816441e-05
    Using multitaper spectrum estimation with 7 DPSS windows
1.5910365528611955e-06
0.12095306191247111
[5.69916435 7.12395543]


In [32]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#alpha
print(total_sum_pds)
psds_alpha, frqs_alpha = mne.time_frequency.psd_multitaper(epochs, fmin=7.5, fmax=12.5, tmin=None, tmax=None)
alpha_sum_pds = np.sum(psds_alpha)
print(alpha_sum_pds)
alpha_ratio = (alpha_sum_pds) / (total_sum_pds)
print(alpha_ratio)
print(frqs_alpha)

1.3154165158816441e-05
    Using multitaper spectrum estimation with 7 DPSS windows
2.377667515958959e-06
0.18075396555024645
[ 8.54874652  9.9735376  11.39832869]


In [33]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#beta
print(total_sum_pds)
psds_beta, frqs_beta = mne.time_frequency.psd_multitaper(epochs, fmin=14.5, fmax=29.5, tmin=None, tmax=None)
beta_sum_pds = np.sum(psds_beta)
print(beta_sum_pds)
beta_ratio = (beta_sum_pds) / (total_sum_pds)
print(beta_ratio)
print(frqs_beta)

1.3154165158816441e-05
    Using multitaper spectrum estimation with 7 DPSS windows
4.328683621007746e-06
0.3290732303225257
[15.67270195 17.09749304 18.52228412 19.94707521 21.3718663  22.79665738
 24.22144847 25.64623955 27.07103064 28.49582173]


In [34]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#gamma
psds_total, frqs_total = mne.time_frequency.psd_multitaper(epochs, fmin=0.5, fmax=55, tmin=None, tmax=None)
total_sum_pds = np.sum(psds_total)
print(total_sum_pds)
psds_gamma, frqs_gamma = mne.time_frequency.psd_multitaper(epochs, fmin=29.5, fmax=45.5, tmin=None, tmax=None)
gamma_sum_pds = np.sum(psds_gamma)
print(gamma_sum_pds)
gamma_ratio = (gamma_sum_pds) / (total_sum_pds)
print(gamma_ratio)
print(frqs_gamma)

    Using multitaper spectrum estimation with 7 DPSS windows
1.3157169067738685e-05
    Using multitaper spectrum estimation with 7 DPSS windows
1.0055525557212787e-06
0.07642620920535928
[29.92061281 31.3454039  32.77019499 34.19498607 35.61977716 37.04456825
 38.46935933 39.89415042 41.3189415  42.74373259 44.16852368]


In [35]:
#evaluate the signal

#formula from paper

#y1: variance score
y1 = 0
#x1: variance
x1 = variance

if x1 < 50:
    y1 = 0.02*np.power(x1,2)
elif x1 >= 50 and x1 < 100:
    y1 = 0.6*x1 + 20
elif x1 >= 100 and x1 < 2000:
    y1 = 100
elif x1 >= 2000 and x1 < 5000:
    y1 = -0.013333*x1 + 126.6
elif x1 >= 5000 and x1 < 10000:
    y1 = 0.006*x1 + 90
else:
    y1 = 15/0.02*np.power((x1-10000),2)
    
#y2: power voltage score
y2 = 0
#x2: signal that are 50Hz (CN) / Total
x2 = power_ratio


if x2 < 0.01:
    y2 = 1
elif x2 >= 0.01 and x2 < 0.1:
    y2 = 1 - 24.691*np.power((x2-0.01),2)
elif x2 >= 0.1 and x2 < 5:
    y2 = 0.8 - 0.00833*np.power((x2-0.1),2)
elif x2 >= 5 and x2 < 10:
    y2 = 0.9 - np.power(0.06,2)
else:
    y2 = 30 / np.power(x2,2)
    
    
#y3: alpha score
y3 = 0
#x3: alpha wave / Total
x3 = delta_ratio

if x3 < 0.35:
#   y3 = 2.8 * np.power(x3,2)
    y3 = 4.375 * np.power(x3,2)
else:
#   y3 = 1.2 * np.power(x3,2) + 2.4*x3 - 0.2
    y3 = 0.376176*np.power(x3,2) + 0.623824

print('x1: ' + str(x1))
print('x2: ' + str(x2))
print('x3: ' + str(x3))

print('y1: ' + str(y1))
print('y2: ' + str(y2))
print('y3: ' + str(y3))

print((y1*y2)*y3)

x1: 185.9692319696723
x2: 0.00022836180677194269
x3: 0.1199612586191608
y1: 100
y2: 1
y3: 0.06295932811653265
6.295932811653265
